##### Initialization: libraries and initial settings

In [1]:
# ----- Load libraries -----

using Dates;
using JLD;
using Distributions;
using DataFrames;
using Colors;

# PlotlyJS
using PlotlyJS;
using ORCA;
pltjs = PlotlyJS

include("./../../code/JuSSM/JuSSM.jl");
using Main.JuSSM;

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-1689383731633845463\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-12257813490409440398\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-1689383731633845463\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
res = jldopen("./results/res.jld");

In [3]:
# ----- Load results from output file -----

nDraws        = read(res, "nDraws");
burnin        = read(res, "burnin");
σʸ            = permutedims(read(res, "σʸ"));
date          = read(res, "date");
data          = read(res, "data"); #.* σʸ';
distr_α       = read(res, "distr_α");
chain_θ_bound = read(res, "chain_θ_bound"); # d, Z, R, c, T, Q, λ, ρ, total
MNEMONIC      = read(res, "MNEMONIC");

data[ismissing.(data)] .= NaN;

par_ind  = read(res, "par_ind");

par_size = SizeParSsm(sum(par_ind.d),
                                 sum(sum(par_ind.Z)),
                                 sum(sum(par_ind.R)),
                                 sum(par_ind.c),
                                 sum(sum(par_ind.T)),
                                 sum(sum(par_ind.Q)),
                                 sum(sum(par_ind.λ)),
                                 sum(sum(par_ind.ρ)),
                                 sum(par_ind.d) + sum(  sum(par_ind.Z)) + sum(sum(par_ind.R)) +
                                    sum(par_ind.c) + sum(sum(par_ind.T)) + sum(sum(par_ind.Q)) +
                                    sum(sum(par_ind.λ)) + sum(sum(par_ind.ρ)));

# ----- Set titles and scales -----

titles = ["CBO: Real GDP cycle", "Real GDP", "SPF: Expected GDP", "Unemployment rate", "Employment", "Oil price", "CPI inflation", "SPF: Expected inflation", "UoM: Expected inflation"];
scales = ["Bil. Chn. 2009\$", "Bil. Chn. 2009\$", "Bil. Chn. 2009\$", "Percent", "Thous.", "\$/Barrel", "Percent", "Percent", "Percent"];

In [4]:
# ----- Get trends and cycles -----

n          = size(data)[2];
TT         = size(distr_α, 2);
ind_trends = [10; collect(15:3:size(distr_α)[1])[1:3]; 28] # There is no index for inflation, please use if-else
ind_cycles = [8; 8; 11; collect(13:3:size(distr_α)[1])[1:4]; 24; 26]; # The first two entries are equal (that's fine)

In [5]:
# The first coefficient in chain_θ_bound is the intercept (differencial) for GDP SPF
# Thus, starting from id 2 is correct

Zᵖ = [ones(2, nDraws-burnin); 
      chain_θ_bound[2:4, burnin+1:end];
      zeros(1, nDraws-burnin);
      chain_θ_bound[5:7, burnin+1:end]];
 
Zᵖ₂ = [zeros(2, nDraws-burnin); 
       chain_θ_bound[8:10, burnin+1:end];
       zeros(1, nDraws-burnin);
       chain_θ_bound[11:13, burnin+1:end]];

Zᵖ₃ = [zeros(2, nDraws-burnin); 
       chain_θ_bound[14:16, burnin+1:end];
       zeros(1, nDraws-burnin);
       chain_θ_bound[17:19, burnin+1:end]];

Zᵖ₄ = [zeros(2, nDraws-burnin); 
       chain_θ_bound[20:22, burnin+1:end];
       zeros(1, nDraws-burnin);
       chain_θ_bound[23:25, burnin+1:end]];

Zᵉ = [zeros(5, nDraws-burnin); 
      ones(1, nDraws-burnin); 
      chain_θ_bound[26:28, burnin+1:end]];

Zᵗ = [zeros(6, nDraws-burnin); 
      (1 ./ σʸ[end-2:end]).*ones(3, nDraws-burnin)];

Z_GDP = [zeros(1, nDraws-burnin);
         ([1;3] ./ σʸ[2:3]).*ones(2, nDraws-burnin);
         zeros(6, nDraws-burnin)];

In [6]:
PC = zeros(n, TT, nDraws-burnin);
EP = zeros(n, TT, nDraws-burnin);
T  = zeros(n, TT, nDraws-burnin);
T_GDP = zeros(n, TT, nDraws-burnin);

for i=1:nDraws-burnin   
    PC[:, :, i] = (Zᵖ[:, i] .* distr_α[1, :, i]') .+ 
                  (Zᵖ₂[:, i] .* distr_α[2, :, i]') .+
                  (Zᵖ₃[:, i] .* distr_α[3, :, i]') .+
                  (Zᵖ₄[:, i] .* distr_α[4, :, i]');
    
    EP[:, :, i] = Zᵉ[:, i] .* distr_α[5, :, i]';
    T[:, :, i]  = Zᵗ[:, i] .* distr_α[7, :, i]';
    T_GDP[:, :, i] = Z_GDP[:, i] .* distr_α[10, :, i]';
    T_GDP[3, :, i] .+= chain_θ_bound[1,burnin+i];
end

In [7]:
iC = distr_α[ind_cycles, :, :];
iT = distr_α[ind_trends, :, :];

PCᵐ = median(PC, dims=3)[:, :, 1]' .* σʸ;
EPᵐ = median(EP, dims=3)[:, :, 1]' .* σʸ;
Tᵐ  = median(T, dims=3)[:, :, 1]' .* σʸ;
T_GDPᵐ = median(T_GDP, dims=3)[:, :, 1]' .* σʸ;
iCᵐ = median(iC, dims=3)[:, :, 1]' .* σʸ;
iTᵐ = median(iT, dims=3)[:, :, 1]' .* σʸ[[2,4,5,6,9]]';

In [8]:
PC95 = zeros(size(PCᵐ));
PC05 = zeros(size(PCᵐ));
PC84 = zeros(size(PCᵐ));
PC16 = zeros(size(PCᵐ));

EP95 = zeros(size(EPᵐ));
EP05 = zeros(size(EPᵐ));
EP84 = zeros(size(EPᵐ));
EP16 = zeros(size(EPᵐ));

T95 = zeros(size(Tᵐ));
T05 = zeros(size(Tᵐ));
T84 = zeros(size(Tᵐ));
T16 = zeros(size(Tᵐ));

T_GDP95 = zeros(size(T_GDPᵐ));
T_GDP05 = zeros(size(T_GDPᵐ));
T_GDP84 = zeros(size(T_GDPᵐ));
T_GDP16 = zeros(size(T_GDPᵐ));

iC95 = zeros(size(iCᵐ));
iC05 = zeros(size(iCᵐ));
iC84 = zeros(size(iCᵐ));
iC16 = zeros(size(iCᵐ));

iT95 = zeros(size(iTᵐ));
iT05 = zeros(size(iTᵐ));
iT84 = zeros(size(iTᵐ));
iT16 = zeros(size(iTᵐ));
iTᵐ[:, 1] = copy(T_GDPᵐ[:, 2]);

for i=1:size(data, 2)
    for j=1:size(data, 1)
        
        PC95[j, i] = quantile(PC[i, j, :], 0.95) .* σʸ[i];
        PC05[j, i] = quantile(PC[i, j, :], 0.05) .* σʸ[i];
        PC84[j, i] = quantile(PC[i, j, :], 0.84) .* σʸ[i];
        PC16[j, i] = quantile(PC[i, j, :], 0.16) .* σʸ[i];
        
        EP95[j, i] = quantile(EP[i, j, :], 0.95) .* σʸ[i];
        EP05[j, i] = quantile(EP[i, j, :], 0.05) .* σʸ[i];
        EP84[j, i] = quantile(EP[i, j, :], 0.84) .* σʸ[i];
        EP16[j, i] = quantile(EP[i, j, :], 0.16) .* σʸ[i];

        T95[j, i] = quantile(T[i, j, :], 0.95) .* σʸ[i];
        T05[j, i] = quantile(T[i, j, :], 0.05) .* σʸ[i];
        T84[j, i] = quantile(T[i, j, :], 0.84) .* σʸ[i];
        T16[j, i] = quantile(T[i, j, :], 0.16) .* σʸ[i];

        T_GDP95[j, i] = quantile(T_GDP[i, j, :], 0.95) .* σʸ[i];
        T_GDP05[j, i] = quantile(T_GDP[i, j, :], 0.05) .* σʸ[i];
        T_GDP84[j, i] = quantile(T_GDP[i, j, :], 0.84) .* σʸ[i];
        T_GDP16[j, i] = quantile(T_GDP[i, j, :], 0.16) .* σʸ[i];

        iC95[j, i] = quantile(iC[i, j, :], 0.95) .* σʸ[i];
        iC05[j, i] = quantile(iC[i, j, :], 0.05) .* σʸ[i];
        iC84[j, i] = quantile(iC[i, j, :], 0.84) .* σʸ[i];
        iC16[j, i] = quantile(iC[i, j, :], 0.16) .* σʸ[i];
        
        if i == 2
            iT95[j, i-1] = copy(T_GDP95[j, i]);
            iT05[j, i-1] = copy(T_GDP05[j, i]);
            iT84[j, i-1] = copy(T_GDP84[j, i]);
            iT16[j, i-1] = copy(T_GDP16[j, i]);
            
        elseif i>3 && i<=6
            iT95[j, i-2] = quantile(iT[i-2, j, :], 0.95) .* σʸ[i];
            iT05[j, i-2] = quantile(iT[i-2, j, :], 0.05) .* σʸ[i];
            iT84[j, i-2] = quantile(iT[i-2, j, :], 0.84) .* σʸ[i];
            iT16[j, i-2] = quantile(iT[i-2, j, :], 0.16) .* σʸ[i];
            
        elseif i == 9
            iT95[j, i-4] = quantile(iT[i-4, j, :], 0.95) .* σʸ[i];
            iT05[j, i-4] = quantile(iT[i-4, j, :], 0.05) .* σʸ[i];
            iT84[j, i-4] = quantile(iT[i-4, j, :], 0.84) .* σʸ[i];
            iT16[j, i-4] = quantile(iT[i-4, j, :], 0.16) .* σʸ[i];
        end
    end
end

In [9]:
# ----- Dates: add h dates to date -----

max_h = size(PCᵐ)[1] - size(date)[1];

for hz=1:max_h
    
    last_month = Dates.month(date[end]);
    last_year  = Dates.year(date[end]);
    new_month  = copy(last_month) + 1;
    new_year   = copy(last_year);
    
    if last_month + 1 > 12
        new_year  += 1;
        new_month  = 1;
    end
    
    new_entry = Dates.lastdayofquarter(Date(new_year, new_month, 1));
    date      = vcat(date, DateTime(new_entry));
end

---

##### Historical decompositions

<i>Cycles</i>

In [10]:
figures = Array{Any}(undef, 9);

c1 = "rgba(0, 48, 158, .75)"; #"rgba(0, 0, 158, .7)";
c2 = "rgba(255, 0, 0, .75)";
c3 = "rgba(255, 190, 0, .75)";

for i=1:9
    trace1 = pltjs.bar(;x=date[1:end-max_h], y=PCᵐ[1:end-max_h, i], name="Business cycle", marker_color=c1, showlegend=i==1);
    trace2 = pltjs.bar(x=date[1:end-max_h], y=EPᵐ[1:end-max_h, i], name="Energy price cycle", marker_color=c2, showlegend=i==1);
    trace3 = pltjs.bar(x=date[1:end-max_h], y=iCᵐ[1:end-max_h, i], name="Idiosyncratic cycle", marker_color=c3, showlegend=i==1);
    trace4 = pltjs.scatter(x=date[1:end-max_h], y=PCᵐ[1:end-max_h, i]+EPᵐ[1:end-max_h, i]+iCᵐ[1:end-max_h, i], name="Total cycle", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==1)
    
    databar = [trace1, trace2, trace3, trace4];
    
        layout  = pltjs.Layout(;title=titles[i], titlefont_size=10,
                               xaxis=attr(tickfont_size=14, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=-90),
                               yaxis=attr(tickfont_size=14, showgrid=true, linecolor="black", mirror=true, titlefont=attr(size=14), title=scales[i]),
                               barmode="relative", 
                               bargap=0,
                               bargroupgap=0);
        

    figures[i] = pltjs.plot(databar, layout);
end

fig = [figures[1] figures[2] figures[3]; figures[4] figures[5] figures[6]; figures[7] figures[8] figures[9]];

# Bars
fig.plot.layout["barmode"] = "relative";
fig.plot.layout["bargap"]  = 0.02;

# Size
fig.plot.layout["width"]  = 1000;
fig.plot.layout["height"] = 800;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:size(data, 2)
    fig.plot.layout["annotations"][i][:font][:size] = 20;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.08, x=0.065, font=attr(size=14))

pltjs.savefig(fig, "./img/historical_decomposition.pdf", format="pdf")

"./img/historical_decomposition.pdf"

In [11]:
figures = Array{Any}(undef, 9);

c1 = "rgba(0, 48, 158, .75)"; #"rgba(0, 0, 158, .7)";
c2 = "rgba(255, 0, 0, .75)";
c3 = "rgba(255, 190, 0, .75)";

for i=1:9
    trace1 = pltjs.bar(;x=date[1:end], y=PCᵐ[1:end, i], name="Business cycle", marker_color=c1, showlegend=i==1);
    trace2 = pltjs.bar(x=date[1:end], y=EPᵐ[1:end, i], name="Energy price cycle", marker_color=c2, showlegend=i==1);
    trace3 = pltjs.bar(x=date[1:end], y=iCᵐ[1:end, i], name="Idiosyncratic cycle", marker_color=c3, showlegend=i==1);
    trace4 = pltjs.scatter(x=date[1:end], y=PCᵐ[1:end, i]+EPᵐ[1:end, i]+iCᵐ[1:end, i], name="Total cycle", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==1)
    
    databar = [trace1, trace2, trace3, trace4];
    
        layout  = pltjs.Layout(;title=titles[i], titlefont_size=10,
                               xaxis=attr(tickfont_size=14, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=-90),
                               yaxis=attr(tickfont_size=14, showgrid=true, linecolor="black", mirror=true, titlefont=attr(size=14), title=scales[i]),
                               barmode="relative", 
                               bargap=0,
                               bargroupgap=0);
        

    figures[i] = pltjs.plot(databar, layout);
end

fig = [figures[1] figures[2] figures[3]; figures[4] figures[5] figures[6]; figures[7] figures[8] figures[9]];

# Bars
fig.plot.layout["barmode"] = "relative";
fig.plot.layout["bargap"]  = 0.02;

# Size
fig.plot.layout["width"]  = 1000;
fig.plot.layout["height"] = 800;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:size(data, 2)
    fig.plot.layout["annotations"][i][:font][:size] = 20;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.08, x=0.065, font=attr(size=14))

pltjs.savefig(fig, "./img/historical_decomposition_fc.pdf", format="pdf")

"./img/historical_decomposition_fc.pdf"

---

##### Trends and cycles

<i>Trends</i>

In [12]:
figures=Array{Any}(undef, 4)


for i=2:5
    if i == 2
        trace0 = pltjs.scatter(;x=date[3:3:end], y=data[3:3:end, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==4);
        trace1 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDPᵐ[1:3:end-max_h, i] .+ T_GDPᵐ[2:3:end-max_h, i] .+ T_GDPᵐ[3:3:end-max_h, i], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDP95[1:3:end-max_h, i] .+ T_GDP95[2:3:end-max_h, i] .+ T_GDP95[3:3:end-max_h, i], name="CI, 90%.", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==4);
        trace3 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDP05[1:3:end-max_h, i] .+ T_GDP05[2:3:end-max_h, i] .+ T_GDP05[3:3:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDP84[1:3:end-max_h, i] .+ T_GDP84[2:3:end-max_h, i] .+ T_GDP84[3:3:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==4);
        trace5 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDP16[1:3:end-max_h, i] .+ T_GDP16[2:3:end-max_h, i] .+ T_GDP16[3:3:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDPᵐ[1:3:end-max_h, i] .+ T_GDPᵐ[2:3:end-max_h, i] .+ T_GDPᵐ[3:3:end-max_h, i], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==4);

    elseif i == 3
        trace0 = pltjs.scatter(;x=date[3:3:end], y=data[3:3:end, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==4);
        trace1 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDPᵐ[3:3:end-max_h, i], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDP95[3:3:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==4);
        trace3 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDP05[3:3:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDP84[3:3:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==4);
        trace5 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDP16[3:3:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=date[3:3:end-max_h], y=T_GDPᵐ[3:3:end-max_h, i], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==4);

    else
        trace0 = pltjs.scatter(;x=date[1:end], y=data[1:end, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==4);
        trace1 = pltjs.scatter(;x=date[1:end-max_h], y=iTᵐ[1:end-max_h, i-2], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=date[1:end-max_h], y=iT95[1:end-max_h, i-2], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==4);
        trace3 = pltjs.scatter(;x=date[1:end-max_h], y=iT05[1:end-max_h, i-2], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=date[1:end-max_h], y=iT84[1:end-max_h, i-2], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==4);
        trace5 = pltjs.scatter(;x=date[1:end-max_h], y=iT16[1:end-max_h, i-2], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=date[1:end-max_h], y=iTᵐ[1:end-max_h, i-2], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==4);
     end
    
     layout  = pltjs.Layout(;title=titles[i], titlefont=attr(size=12),
                            xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=-90, zeroline=false),
                            yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title=scales[i]));
     if i != 3
         figures[i-1] = pltjs.plot([trace1; trace2; 
                                  trace1; trace3; 
                                  trace1; trace4;
                                  trace1; trace5; 
                                  trace6; trace0], layout);
    else
         figures[i-1] = pltjs.plot([trace1; trace2; 
                                  trace1; trace3; 
                                  trace1; trace4;
                                  trace1; trace5; 
                                  trace6; trace0], layout);
    end
end


fig = [figures[1] figures[2]; figures[3] figures[4]];

fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 650;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:4
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.10, x=0.285, font=attr(size=10))

pltjs.savefig(fig, "./img/total_trend_real.pdf", format="pdf")

"./img/total_trend_real.pdf"

In [13]:
figures=Array{Any}(undef, 3)


for i=7:9
    
     if i != 8
         trace0 = pltjs.scatter(;x=date[1:end], y=data[1:end, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==7);
     else
         trace0 = pltjs.scatter(;x=date[1:end], y=data[1:end, i], name="Data",mode="markers", marker=attr(size=3, color="black"), showlegend=i==7);
     end
    
    if i != 8
         trace0 = pltjs.scatter(;x=date[1:end], y=data[1:end, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==7);
         trace1 = pltjs.scatter(;x=date[1:end-max_h], y=Tᵐ[1:end-max_h, i], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
         trace2 = pltjs.scatter(;x=date[1:end-max_h], y=T95[1:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==7);
         trace3 = pltjs.scatter(;x=date[1:end-max_h], y=T05[1:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
         trace4 = pltjs.scatter(;x=date[1:end-max_h], y=T84[1:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==7);
         trace5 = pltjs.scatter(;x=date[1:end-max_h], y=T16[1:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
         trace6 = pltjs.scatter(;x=date[1:end-max_h], y=Tᵐ[1:end-max_h, i], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==7);

    else
        trace0 = pltjs.scatter(;x=date[3:3:end], y=data[3:3:end, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==7);
        trace1 = pltjs.scatter(;x=date[3:3:end-max_h], y=Tᵐ[3:3:end-max_h, i], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=date[3:3:end-max_h], y=T95[3:3:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==7);
        trace3 = pltjs.scatter(;x=date[3:3:end-max_h], y=T05[3:3:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=date[3:3:end-max_h], y=T84[3:3:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==7);
        trace5 = pltjs.scatter(;x=date[3:3:end-max_h], y=T16[3:3:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=date[3:3:end-max_h], y=Tᵐ[3:3:end-max_h, i], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==7);
     end
     
        
         layout  = pltjs.Layout(;title=titles[i], titlefont=attr(size=12),
                                xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=-90, zeroline=false, range=[date[1], date[end-max_h]]),
                                yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title=scales[i], range=[-2,8]));
     
         figures[i-6] = pltjs.plot([trace1; trace2; 
                                  trace1; trace3; 
                                  trace1; trace4;
                                  trace1; trace5; 
                                  trace6; trace0], layout);
end

fig = [figures[1]; figures[2]; figures[3]];

fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 800;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:3
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.08, x=0.285, font=attr(size=10))

pltjs.savefig(fig, "./img/common_trend_ci.pdf", format="pdf")

"./img/common_trend_ci.pdf"

##### Parametric spectrum

In [14]:
# ----- Spectral density distribution -----

num_s(ω, λ, ρ, σ²) = σ² .* (1 .+ ρ.^2 .- 2*ρ.*cos.(ω).*cos.(λ));
den_s(ω, λ, ρ)     = (2*pi) .* (1 .+ ρ.^4 .+ 4*(ρ^2).*(cos.(λ).^2) .- 4*ρ.*(1 .+ ρ.^2).*cos.(λ).*cos.(ω) .+ 2*(ρ.^2).*cos.(2*ω));
s(ω, λ, ρ, σ²)     = num_s(ω, λ, ρ, σ²) ./ den_s(ω, λ, ρ);

ω       = collect(0:0.01:pi);
s_distr = zeros(length(ω), 3, nDraws-burnin);

for i=1:3
    
    # parameters
    if i<3
        λ  = chain_θ_bound[par_size.R+par_size.d+par_size.Z+par_size.Q+par_size.c+par_size.T+i, :];
        ρ  = chain_θ_bound[par_size.R+par_size.d+par_size.Z+par_size.Q+par_size.c+par_size.T+par_size.λ+i, :];
        σ² = chain_θ_bound[par_size.R+par_size.d+par_size.Z+i, :];
    else
        λ  = chain_θ_bound[par_size.R+par_size.d+par_size.Z+par_size.Q+par_size.c+par_size.T+2+4, :];
        ρ  = chain_θ_bound[par_size.R+par_size.d+par_size.Z+par_size.Q+par_size.c+par_size.T+par_size.λ+2+4, :];
        σ² = chain_θ_bound[par_size.R+par_size.d+par_size.Z+3+6+1, :];
    end
    
    # spectral density
    for draw=1:nDraws-burnin
        s_distr[:, i, draw] = s(ω[:], λ[draw], ρ[draw], σ²[draw]);
    end
end


In [15]:
# PC spectrum
PCᵐ_s  = median(s_distr[:, 1, :], dims=2);
PC95_s = zeros(size(PCᵐ_s));
PC05_s = zeros(size(PCᵐ_s));
PC84_s = zeros(size(PCᵐ_s));
PC16_s = zeros(size(PCᵐ_s));

EPᵐ_s  = median(s_distr[:, 2, :], dims=2);
EP95_s = zeros(size(EPᵐ_s));
EP05_s = zeros(size(EPᵐ_s));
EP84_s = zeros(size(EPᵐ_s));
EP16_s = zeros(size(EPᵐ_s));

iCᵐ_s  = median(s_distr[:, 3, :], dims=2);
iC95_s = zeros(size(iCᵐ_s));
iC05_s = zeros(size(iCᵐ_s));
iC84_s = zeros(size(iCᵐ_s));
iC16_s = zeros(size(iCᵐ_s));

for i=1:size(s_distr, 1)
    PC95_s[i] = quantile(s_distr[i, 1, :], 0.95);
    PC05_s[i] = quantile(s_distr[i, 1, :], 0.05);
    PC84_s[i] = quantile(s_distr[i, 1, :], 0.84);
    PC16_s[i] = quantile(s_distr[i, 1, :], 0.16);
    
    EP95_s[i] = quantile(s_distr[i, 2, :], 0.95);
    EP05_s[i] = quantile(s_distr[i, 2, :], 0.05);
    EP84_s[i] = quantile(s_distr[i, 2, :], 0.84);
    EP16_s[i] = quantile(s_distr[i, 2, :], 0.16);
    
    iC95_s[i] = quantile(s_distr[i, 1, :], 0.95);
    iC05_s[i] = quantile(s_distr[i, 1, :], 0.05);
    iC84_s[i] = quantile(s_distr[i, 1, :], 0.84);
    iC16_s[i] = quantile(s_distr[i, 1, :], 0.16);
end

In [16]:
# ----- Chart -----

figures=Array{Any}(undef, 4)

titles_sub = ["Business cycle", "Energy price cycle", "Business cycle: spectral density", "Energy price cycle: spectral density"];

for i=1:4
    
    if i == 1 || i == 2
        layout  = pltjs.Layout(;title=titles_sub[i], titlefont=attr(size=12),
                               xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false),
                               yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="Percent"));
    else
        layout  = pltjs.Layout(;title=titles_sub[i], titlefont=attr(size=12),
                               xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false, titlefont=attr(size=10), title="Frequency"),
                               yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="Spectral density"));
    end
    
    if i == 1
        trace1  = pltjs.scatter(;x=date[1:end-max_h], y=PCᵐ[1:end-max_h, 1], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2  = pltjs.scatter(;x=date[1:end-max_h], y=PC95[1:end-max_h, 1], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==1);
        trace3  = pltjs.scatter(;x=date[1:end-max_h], y=PC05[1:end-max_h, 1], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4  = pltjs.scatter(;x=date[1:end-max_h], y=PC84[1:end-max_h, 1], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==1);
        trace5  = pltjs.scatter(;x=date[1:end-max_h], y=PC16[1:end-max_h, 1], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6a = pltjs.scatter(;x=date[1:end-max_h], y=PCᵐ[1:end-max_h, 1], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==1);
        trace6b = pltjs.scatter(;x=date[1:end-max_h], y=PCᵐ[1:end-max_h, 1], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        trace7  = pltjs.scatter(;x=date[1:end-max_h], y=zeros(size(PCᵐ)[1]-max_h), name="", mode="lines", line=attr(width=1, color="black"), showlegend=false);
        figures[i] = pltjs.plot([trace6a; trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6b; trace7], layout);
    elseif i == 2
        trace1 = pltjs.scatter(;x=date[1:end-max_h], y=EPᵐ[1:end-max_h, 6], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=date[1:end-max_h], y=EP95[1:end-max_h, 6], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace3 = pltjs.scatter(;x=date[1:end-max_h], y=EP05[1:end-max_h, 6], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=date[1:end-max_h], y=EP84[1:end-max_h, 6], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace5 = pltjs.scatter(;x=date[1:end-max_h], y=EP16[1:end-max_h, 6], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=date[1:end-max_h], y=EPᵐ[1:end-max_h, 6], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        trace7 = pltjs.scatter(;x=date[1:end-max_h], y=zeros(size(EPᵐ)[1]-max_h), name="", mode="lines", line=attr(width=1, color="black"), showlegend=false);
        figures[i] = pltjs.plot([trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6; trace7], layout);
    elseif i == 3
        trace1 = pltjs.scatter(;x=ω, y=PCᵐ_s[:], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=ω, y=PC95_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace3 = pltjs.scatter(;x=ω, y=PC05_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=ω, y=PC84_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace5 = pltjs.scatter(;x=ω, y=PC16_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=ω, y=PCᵐ_s[:], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        figures[i] = pltjs.plot([trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6], layout);

    else
        trace1 = pltjs.scatter(;x=ω, y=EPᵐ_s[:], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=ω, y=EP95_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace3 = pltjs.scatter(;x=ω, y=EP05_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=ω, y=EP84_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace5 = pltjs.scatter(;x=ω, y=EP16_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=ω, y=EPᵐ_s[:], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        figures[i] = pltjs.plot([trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6], layout);

    end
end


fig = [figures[1] figures[2]; figures[3] figures[4]];

fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 600;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:4
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.165, x=0.33, font=attr(size=10), traceorder="normal")

savefig(fig, "./img/cycles_spectral.pdf", format="pdf")

"./img/cycles_spectral.pdf"

In [17]:
figures = Array{Any}(undef,1)
trace1 = pltjs.scatter(;x=date[1:end], y=100*(PCᵐ[:,2] .+ iCᵐ[:,2])./iTᵐ[:,1], line=attr(color="red"), name="Output gap");

layout = pltjs.Layout(;titlefont=attr(size=14), width=800, height=300)
fig = pltjs.plot(trace1, layout)
pltjs.savefig(fig, "./img/output_gap_monthly.pdf", format="pdf")

"./img/output_gap_monthly.pdf"

In [18]:
quarterly_cycle = (PCᵐ[1:3:end,2]+iCᵐ[1:3:end,2]+PCᵐ[2:3:end,2]+iCᵐ[2:3:end,2]+PCᵐ[3:3:end,2]+iCᵐ[3:3:end,2]);
quarterly_trend = (iTᵐ[1:3:end,1]+iTᵐ[2:3:end,1]+iTᵐ[3:3:end,1]);

figures = Array{Any}(undef, 1)
trace1 = pltjs.scatter(;x=date[3:3:end-max_h], y=100*quarterly_cycle./quarterly_trend, line=attr(color="red"), name="Output gap");

layout = pltjs.Layout(;titlefont=attr(size=14), width=800, height=300)
fig = pltjs.plot([trace1], layout)
pltjs.savefig(fig, "./img/output_gap_quarterly.pdf", format="pdf")

"./img/output_gap_quarterly.pdf"

In [19]:
using CSV;
CSV.write("./results_csv/output_gap_quarterly.csv", DataFrame(x=100*quarterly_cycle./quarterly_trend))
CSV.write("./results_csv/output_gap_monthly.csv", DataFrame(x=(PCᵐ[:,2]+iCᵐ[:,2])./iTᵐ[:,1]))
CSV.write("./results_csv/output_trend_quarterly.csv", DataFrame(x=quarterly_trend))
CSV.write("./results_csv/inflation_trend_monthly.csv", DataFrame(x=Tᵐ[:,7]))

"./results_csv/inflation_trend_monthly.csv"